In [1]:
# import dask.dataframe as dd

In [2]:
# df= dd.read_csv('final_train.csv')

In [1]:
from multiprocessing import Process, Lock, Manager

In [2]:
import sys
import numpy as np
import matplotlib as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import *
from math import sqrt

In [6]:
import pandas as pd
from smart_open import smart_open
import os
from configparser import ConfigParser

config = ConfigParser()

config_file = os.path.join(os.path.dirname('__file__'), 'config.ini')

config.read(config_file)
default = config['aws.data']
aws_key = default['accessKey']
aws_secret = default['secretAccessKey']

bucket_name = 'pubg-dataset-files'
object_key = 'final_train.csv'

path = 's3://{}:{}@{}/{}'.format(aws_key, aws_secret, bucket_name, object_key)

df_train = pd.read_csv(smart_open(path))

In [7]:
X=df_train.drop('winPlacePerc',axis=1)
y=df_train.winPlacePerc

In [26]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

error_metric = pd.DataFrame({'r2_train': [],
                            'r2_test': [],
                             'rms_train':[], 
                            'rms_test': [],
                            'mae_train': [],
                            'mae_test':[]})
    
rmse_dict = {} 

In [27]:
def calc_error_metric(modelname, model, X_train_scale, y_train, X_test_scale, y_test):
    global error_metric
    y_train_predicted = model.predict(X_train_scale)
    y_test_predicted = model.predict(X_test_scale)
        
    #MAE, RMS, MAPE, R2
    
    r2_train = r2_score(y_train, y_train_predicted)
    r2_test = r2_score(y_test, y_test_predicted)
    
    rms_train = sqrt(mean_squared_error(y_train, y_train_predicted))
    rms_test = sqrt(mean_squared_error(y_test, y_test_predicted))
        
    mae_train = mean_absolute_error(y_train, y_train_predicted)
    mae_test = mean_absolute_error(y_test, y_test_predicted)
        
#     mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
#     mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
        
    rmse_dict[modelname] = rms_test
        
    df_local = pd.DataFrame({'Model':[modelname],
                            'r2_train': [r2_train],
                            'r2_test': [r2_test],
                            'rms_train':[rms_train], 
                            'rms_test': [rms_test],
                            'mae_train': [mae_train],
                            'mae_test': [mae_test]})
        
    error_metric = pd.concat([error_metric, df_local])
    return error_metric

# Now let's predict using different models and check the accuracy

## 1. Linear Regression

In [28]:
def linear_regress(X_train, y_train, X_test, y_test, return_dict_logred):
    lm=LinearRegression()
    lm.fit(X_train,y_train)
    y_train_pred=lm.predict(X_train)
    #calc_error_metric('Linear Regression',lm, X_train, y_train, X_test, y_test)

## 2. Random Forest Regressor

In [29]:
def random_forest_regress(X_train, y_train, X_test, y_test, return_dict_rf):
    rf = RandomForestRegressor(n_estimators=30, max_depth=7)
    rf.fit(X_train, y_train)
    y_train_pred=rf.predict(X_train)
    calc_error_metric('RandomForest', rf, X_train, y_train, X_test, y_test)

## 3. Neural Network

In [30]:
def neural_net(X_train, y_train, X_test, y_test, return_dict_nn):
    nn = MLPRegressor()
    nn.fit(X_train, y_train)
    y_train_pred=nn.predict(X_train)
    calc_error_metric('Nueral Network', nn, X_train, y_train, X_test, y_test)
    

In [31]:


manager = Manager()
return_dict_logred = manager.dict()
return_dict_rf = manager.dict()
return_dict_nn = manager.dict()

In [32]:
import multiprocessing as mp
P1=Process(target=linear_regress,args=(X_train, y_train,X_test,y_test,return_dict_logred))
#processes.append(mp.Process(target=random_forest_regress,args=(X_train, y_train,X_test,y_test,return_dict_rf)))
#processes.append(mp.Process(target=neural_net,args=(X_train, y_train,X_test,y_test,return_dict_nn)))


P1.start()

P1.join()



BrokenPipeError: [Errno 32] Broken pipe